In [43]:
import kfp
import os
import requests

from kfp.dsl import Input, Model, component, Dataset, Output, Artifact
from kfp.dsl import InputPath, OutputPath, pipeline, component, PipelineTask
from kfp.components import load_component_from_file

In [44]:
@component
def make_artifact(code_lines: str, output_file: Output[Artifact]):
    import os
    os.makedirs(output_file.path, exist_ok=True)
    output_file_path = os.path.join(output_file.path, "job.py")

    with open(output_file_path, 'w') as f:
        f.write(code_lines)

In [45]:
@component(
    base_image="rayproject/ray:2.41.0"
)
def submit_job(code_dir: Input[Artifact], host: str, port: int):
    import os
    import shutil
    from ray.job_submission import JobSubmissionClient

    target_dir = "./code"
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    shutil.copytree(code_dir.path, target_dir)

    client = JobSubmissionClient(f"http://{host}:{port}")

    job_id = client.submit_job(
        entrypoint="python job.py",
        runtime_env={
            "working_dir": target_dir,
            "pip": [
                "deepspeed",
                "torch",
                "datasets",
                "transformers",
                "torchmetrics",
                "ray[train]"
            ]
        }
    )
    
    print(job_id)

In [46]:
with open('./ray-job.py', 'r') as f:
    code = f.read()

In [47]:
@pipeline(name='ray-job-submit-pipeline')
def file_processing_pipeline(host: str, port: int):
    code_op = make_artifact(code_lines=code)
    job_op = submit_job(code_dir=code_op.output, host=host, port=port)

In [48]:
kfp.compiler.Compiler().compile(file_processing_pipeline, './pipelines-yaml/file_processing_pipeline.yaml')

In [50]:
client = kfp.Client()
run = client.create_run_from_pipeline_func(file_processing_pipeline, arguments={"host": "raycluster-kuberay-head-svc.demo-ns.svc.cluster.local", "port": 8265}, enable_caching=False)